# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:

#pip install gmaps
city_df=pd.read_csv("citydatainfo.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
gmaps.configure(api_key=g_key)


In [43]:
location=city_df[["Latitude","Longitude"]]
weight=city_df["Humidity"]
fig=gmaps.figure()
heatlayer=gmaps.heatmap_layer(location,weights=weight,dissipating=False,max_intensity=400,point_radius=5)
fig.add_layer(heatlayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [44]:
perfect_weather=city_df[(city_df["Temperature"] >= 70) 
                        &(city_df["Temperature"] <= 85)
                        &(city_df["Wind Speed"] <= 10)
                        &(city_df["Cloudiness"] == 0)]

perfect_weather.head()


hotel_df= perfect_weather

hotel_df.insert(3,"Hotel","")

hotel_df.head()

,Unnamed: 0,City,Country,Hotel,Date,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed
17,17,rocha,UY,,1619723942,-54.3333,-34.4833,72.28,49,0,2.08
25,25,saint-pierre,RE,,1619723946,55.4781,-21.3393,72.45,78,0,6.91
82,82,beloha,MG,,1619723968,45.0500,-25.1667,73.81,77,0,8.21
87,87,saint-philippe,RE,,1619723970,55.7679,-21.3585,71.60,78,0,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
target_coordinates= "-54.3333,-34.4833 "
target_radius=5000
target_type="lodgings"

params= {
    "location":target_coordinates,
    "radius":target_radius,
    "type":target_type,
    
}

In [48]:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response= requests.get(base_url,params=params)
print(response.url)

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-54.3333%2C-34.4833+&radius=5000&type=lodgings


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
